In [1]:
import pandas as pd
from pandas.io.json import json_normalize
import json
from copy import copy
idx = pd.IndexSlice

In [2]:
with open('../data/all_declarations.json', 'r') as f:
    data = json.load(f)

In [7]:
data.keys()

dict_keys(['data'])

In [11]:
data['data'][0]

{'date': '31/_12/2016г.',
 'deputy': {'income': ['997 146 с. -по месту основной работы',
   '714 546 с.-пенсия'],
  'other_estate': [],
  'real_estate': [{'наименование': 'дом', 'площадь (кв.м)': 172}],
  'spendeturs': []},
 'relatives': {'income': [],
  'other_estate': [{'наименование': 'лег/автомобиль ',
    'сведения о транспортных средствах': 'Тойота Камри, 2015г. V=2494Б'}],
  'real_estate': [{'наименование': 'квартира', 'площадь (кв.м)': 187}],
  'spendeturs': []},
 'Должность декларанта': 'Депутат',
 'Наименование государственного/ муниципального органа': 'Жогорку Кенеш Кыргызской Республики',
 'ФИО декларанта': 'Абдылдаев Мыктыбек Юсупович'}

## Разделим данные на отдельные, "Tidy" таблички

всего будут следующие таблички:
    - депутаты
    - income/spendings
    - real estate
    - other estate

### 1. Deputies

In [8]:
deputies_cols = ['Должность декларанта', 'Наименование государственного/ муниципального органа', 'ФИО декларанта']

In [9]:
deputies = pd.DataFrame([{k:el[k] for k in deputies_cols} for el in data['data']])

In [10]:
deputies.head(2)

,Должность декларанта,Наименование государственного/ муниципального органа,ФИО декларанта
0,Депутат,Жогорку Кенеш Кыргызской Республики,Абдылдаев Мыктыбек Юсупович
1,Депутат,Жогорку Кенеш Кыргызской Республики,Абжалиев Алиярбек Токобекович


## Income / Spendings

In [23]:
money = []
for dep in data['data']:
    name = dep['ФИО декларанта']
    for p in 'deputy', 'relatives':
        if p in dep:
            
            for tp in 'income', 'spendeturs':
                if len(dep[p][tp])>0:
                    records = pd.DataFrame(dep[p][tp])
                    records.columns = ['record']
                    records['type'] = tp
                    records['person'] = p
                    records['name'] = name
                    money.append(records)

money = pd.concat(money)

In [25]:
money.head(10)

,record,type,person,name
0,997 146 с. -по месту основной работы,income,deputy,Абдылдаев Мыктыбек Юсупович
1,714 546 с.-пенсия,income,deputy,Абдылдаев Мыктыбек Юсупович
0,787 075 с. - по месту основной работы,income,deputy,Абжалиев Алиярбек Токобекович
0,597 633 с. - по месту основной работы,income,deputy,Айнакулов Мухтарбек Айнакулович
0,786 243 с. - по месту основной работы,income,deputy,Айдаров Салайдин Абдираевич
1,198 336 с. - пенсия,income,deputy,Айдаров Салайдин Абдираевич
2,1 000 000 с.-от гражданско-правовых договоров,income,deputy,Айдаров Салайдин Абдираевич
0,852 417 с. - по месту основной работы,income,deputy,Акматов Алмасбек Жумабекович
0,648 163 с. - по месту основной работы,income,deputy,Акаев Жанарбек Кубанычович
0,885 352 с. - по месту основной работы,income,deputy,Алимбеков Нурбек Каарыевич


In [26]:
## this is not tidy yet - we'll address that later

In [54]:
money['amount'] = money.record.str.extract(r'^([\d|\s|,]+)\S+').str.replace(',','').str.replace(' ','').astype(int)

/Users/philippk/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  if __name__ == '__main__':


In [63]:
money['currency'] = money.record.str.extract(r'^[\d|\s|,]+([^-]*)-').str.strip()

/Users/philippk/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  if __name__ == '__main__':


In [64]:
money['detail'] = money.record.str.extract(r'\-(.*)').str.strip()

/Users/philippk/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  if __name__ == '__main__':


In [65]:
money.head(10)

,record,type,person,name,amount,currency,detail
0,997 146 с. -по месту основной работы,income,deputy,Абдылдаев Мыктыбек Юсупович,997146,с.,по месту основной работы
1,714 546 с.-пенсия,income,deputy,Абдылдаев Мыктыбек Юсупович,714546,с.,пенсия
0,787 075 с. - по месту основной работы,income,deputy,Абжалиев Алиярбек Токобекович,787075,с.,по месту основной работы
0,597 633 с. - по месту основной работы,income,deputy,Айнакулов Мухтарбек Айнакулович,597633,с.,по месту основной работы
0,786 243 с. - по месту основной работы,income,deputy,Айдаров Салайдин Абдираевич,786243,с.,по месту основной работы
1,198 336 с. - пенсия,income,deputy,Айдаров Салайдин Абдираевич,198336,с.,пенсия
2,1 000 000 с.-от гражданско-правовых договоров,income,deputy,Айдаров Салайдин Абдираевич,1000000,с.,от гражданско-правовых договоров
0,852 417 с. - по месту основной работы,income,deputy,Акматов Алмасбек Жумабекович,852417,с.,по месту основной работы
0,648 163 с. - по месту основной работы,income,deputy,Акаев Жанарбек Кубанычович,648163,с.,по месту основной работы
0,885 352 с. - по месту основной работы,income,deputy,Алимбеков Нурбек Каарыевич,885352,с.,по месту основной работы


## Real Estate

In [28]:
re = []
for dep in data['data']:
    name = dep['ФИО декларанта']
    for p in 'deputy', 'relatives':
        if p in dep:
            tp = 'real_estate'
            if len(dep[p][tp])>0:
                records = pd.DataFrame(dep[p][tp])
                records['type'] = tp
                records['person'] = p
                records['name'] = name
                re.append(records)

re = pd.concat(re)

In [30]:
re.head(3)

,name,person,type,доля (%),наименование,площадь (кв.м),хозяйствующий субъект
0,Абдылдаев Мыктыбек Юсупович,deputy,real_estate,NaN,дом,172,NaN
0,Абдылдаев Мыктыбек Юсупович,relatives,real_estate,NaN,квартира,187,NaN
0,Айнакулов Мухтарбек Айнакулович,relatives,real_estate,NaN,квартира,115,NaN


## Other estate

In [31]:
oe = []

for dep in data['data']:
    name = dep['ФИО декларанта']
    for p in 'deputy', 'relatives':
        if p in dep:
            tp = 'other_estate'
            if len(dep[p][tp])>0:
                records = pd.DataFrame(dep[p][tp])
                records['type'] = tp
                records['person'] = p
                records['name'] = name
                oe.append(records)

oe = pd.concat(oe)

In [71]:
oe.tail(20)

,name,person,type,доля (%),наименование,сведения о транспортных средствах,хозяйствующий субъект,year
0,Турускулов Жыргалбек Куручбекович,deputy,other_estate,NaN,груз/автомобиль,"МАН 19323, 1995г., V=9973 Д",NaN,1995
0,Турускулов Жыргалбек Куручбекович,relatives,other_estate,NaN,лег/автомобиль,"Тойота Лендкрузер 200, 2016г.,V=4608 Б",NaN,2016
0,Туманбаева Рада Мураталыевна,relatives,other_estate,NaN,груз/автомобиль,"ДАФ- XF 105, 2007г., V=12,9",NaN,2007
1,Туманбаева Рада Мураталыевна,relatives,other_estate,NaN,груз/автомобиль,"ДАФ -XF 95, 2006 г., V=12,6",NaN,NaN
2,Туманбаева Рада Мураталыевна,relatives,other_estate,NaN,лег/автомобиль,"Тойота Матрикс, 2010, V=1,8",NaN,NaN
0,Узакбаев Абылкайыр Осмонбекович,deputy,other_estate,NaN,автомобиль,"РАФ 2203, 1987г.",NaN,1987
0,Шадиев Аскарбек Алимбаевич,deputy,other_estate,50,NaN,NaN,ОсОО,NaN
1,Шадиев Аскарбек Алимбаевич,deputy,other_estate,50,NaN,NaN,ОсОО,NaN
0,Шадиев Аскарбек Алимбаевич,relatives,other_estate,NaN,автомобиль,"Тойота Лендкрузкр 150, 2013г., V3058Б",NaN,2013
0,Шайназаров Тынчтык Урайимович,deputy,other_estate,NaN,автомобиль,Тойота Лендкрузер,NaN,NaN


In [72]:
# oe['year'] = 
oe['year'] = oe['сведения о транспортных средствах'].str.extract(r'(\d+)г\.').str.strip()

/Users/philippk/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  from ipykernel import kernelapp as app


## Store Data

#### Store data as csv's

In [73]:
for df, path in zip([deputies, money, re, oe], ['1_names.csv', '1_money.csv', '1_real_estate.csv', '1_other_property.csv']):
    df.to_csv(f'../data/semi/{path}')

#### Store data as one excel

In [74]:
from pandas import ExcelWriter

def save_xls(df_dict, xls_path):
    writer = ExcelWriter(xls_path)
    for name, df in df_dict.items():
        df.to_excel(writer,name)
    writer.save()

In [75]:
d = {'deputies': deputies,
     'money': money,
     'real_estate': re,
     'other_prop': oe}

save_xls(d, '../data/semi/one_excel_table.xlsx')

#### Store Sqlite

In [76]:
import sqlite3

In [77]:
conn = sqlite3.connect("../data/semi/declarations.db")

In [78]:
for k, df in d.items():
    df.to_sql(k, conn, if_exists='fail')

/Users/philippk/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/core/generic.py:1534: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)


ValueError: Table 'deputies' already exists.

In [47]:
conn.close()